<a href="https://colab.research.google.com/github/EggPudding/Deep-Learning-Practice-with-Codes/blob/main/DenseNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Densely Connected Convolutional Networks (CVPR 2017) Tutorial**

*   Pratice for DenseNet Architecture
*   Orginal Paper: https://arxiv.org/abs/1608.06993
*   Note that you first change **Runtime** to **GPU** setting
*   **CIFAR-10** Dataset is used for practice for simplicity
*   Part of codes from https://github.com/kuangliu/pytorch-cifar

In [1]:
!nvidia-smi # Make Sure you are using GPU

Tue Jan 26 02:20:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### **DenseNet Model Definition**
* In this **Tutorial**, **DenseNet-121** architecture is utilized since relatively simple dataset is used, **CIFAR-10**.


In [4]:
import math

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.backends.cudnn as cudnn
import torch.optim as optim

import os

# Bottleneck Module
# Module output is going to be concatenated with the model input,
# this process is done recursively to construct DenseNet architecture.
class Bottleneck(nn.Module):
    def __init__(self, in_planes, growth_rate):
        super(Bottleneck, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv1 = nn.Conv2d(in_planes, 4*growth_rate, kernel_size=1, bias=False)
        self.bn2 = nn.BatchNorm2d(4*growth_rate)
        self.conv2 = nn.Conv2d(4*growth_rate, growth_rate, kernel_size=3, padding=1, bias=False)

    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = self.conv2(F.relu(self.bn2(out)))
        out = torch.cat([out,x], 1) # Note that DenseNet utilized concatenation rather than skip connection as in ResNet
        return out

# Transition Module
# Lower the resolution of activation map as well as reduce the channels of it.
class Transition(nn.Module):
    def __init__(self, in_planes, out_planes):
        super(Transition, self).__init__()
        self.bn = nn.BatchNorm2d(in_planes)
        self.conv = nn.Conv2d(in_planes, out_planes, kernel_size=1, bias=False)

    def forward(self, x):
        out = self.conv(F.relu(self.bn(x)))
        out = F.avg_pool2d(out, 2)
        return out

# DenseNet Implementation
class DenseNet(nn.Module):
    def __init__(self, block, nblocks, growth_rate=12, reduction=0.5, num_classes=10):
        super(DenseNet, self).__init__()
        self.growth_rate = growth_rate

        num_planes = 2*growth_rate
        self.conv1 = nn.Conv2d(3, num_planes, kernel_size=3, padding=1, bias=False)

        self.dense1 = self._make_dense_layers(block, num_planes, nblocks[0])
        num_planes += nblocks[0]*growth_rate
        out_planes = int(math.floor(num_planes*reduction))
        self.trans1 = Transition(num_planes, out_planes)
        num_planes = out_planes

        self.dense2 = self._make_dense_layers(block, num_planes, nblocks[1])
        num_planes += nblocks[1]*growth_rate
        out_planes = int(math.floor(num_planes*reduction))
        self.trans2 = Transition(num_planes, out_planes)
        num_planes = out_planes

        self.dense3 = self._make_dense_layers(block, num_planes, nblocks[2])
        num_planes += nblocks[2]*growth_rate
        out_planes = int(math.floor(num_planes*reduction))
        self.trans3 = Transition(num_planes, out_planes)
        num_planes = out_planes

        self.dense4 = self._make_dense_layers(block, num_planes, nblocks[3])
        num_planes += nblocks[3]*growth_rate

        self.bn = nn.BatchNorm2d(num_planes)
        self.linear = nn.Linear(num_planes, num_classes)

    def _make_dense_layers(self, block, in_planes, nblock):
        layers = []
        for i in range(nblock):
            layers.append(block(in_planes, self.growth_rate))
            in_planes += self.growth_rate
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.trans1(self.dense1(out))
        out = self.trans2(self.dense2(out))
        out = self.trans3(self.dense3(out))
        out = self.dense4(out)
        out = F.avg_pool2d(F.relu(self.bn(out)), 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def DenseNet121():
    return DenseNet(Bottleneck, [6,12,24,16], growth_rate=32)

### **Hyper Parameter Setting**

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu" # whether using gpu or cpu

model = DenseNet121() # model assignment
model.to(device) # mapping model weight & bias into gpu memory
model = torch.nn.DataParallel(model) # used for parallel setting

cudnn.benchmark = True # using cudnn which optimizes the algorithm

learning_rate = 0.01
batch_size = 128
max_epoch = 10

model_path = 'densenet121_cifar10.pt'

criterion = nn.CrossEntropyLoss() # simple cross-entropy is utilized
optimizer = optim.Adam(model.parameters(), lr=learning_rate) # Adam optimizer utilized

* **torchsummary** is package for visualizing pytorch model.
* Layers, Number of parameters can be viewed through this.

In [6]:
import torchsummary

torchsummary.summary(model.cuda(), (3, 32, 32)) # CIFAR-10 Image has 32 x 32 x 3 shape

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3          [-1, 128, 32, 32]           8,192
       BatchNorm2d-4          [-1, 128, 32, 32]             256
            Conv2d-5           [-1, 32, 32, 32]          36,864
        Bottleneck-6           [-1, 96, 32, 32]               0
       BatchNorm2d-7           [-1, 96, 32, 32]             192
            Conv2d-8          [-1, 128, 32, 32]          12,288
       BatchNorm2d-9          [-1, 128, 32, 32]             256
           Conv2d-10           [-1, 32, 32, 32]          36,864
       Bottleneck-11          [-1, 128, 32, 32]               0
      BatchNorm2d-12          [-1, 128, 32, 32]             256
           Conv2d-13          [-1, 128, 32, 32]          16,384
      BatchNorm2d-14          [-1, 128,

### 학습 (Train) & 검증 (Validation) 함수 정의
___

In [7]:
def train(epoch, max_epoch):
    print(f"Train Epoch [{epoch}/{max_epoch}]")
    model.train() # Model to train mode

    train_loss = 0
    correct = 0
    total = 0
    acc = 0

    for idx, (x, y) in enumerate(train_dataloader):
        x = x.to(device) # maps data into GPU memory
        y = y.to(device)

        optimizer.zero_grad() # reset gradients in optimizer before calculating the loss

        y_pred = model(x) # model inference
        loss = criterion(y_pred, y) # calculating the loss

        loss.backward() # back-propagation to get cumulative gradients

        optimizer.step() # update model parameters
        train_loss += loss.item()
        _, inference = y_pred.max(1)

        total += x.size(0)
        correct += inference.eq(y).sum().item()

        if idx % 100 == 0:
            print(f"Epoch [{epoch}/{max_epoch}] Batch [{idx}] Train Loss: {loss.item()}")

    acc = 100*correct/total
    print(f"Epoch [{epoch}/{max_epoch}] Train Loss: {train_loss/total} Train Accuracy: {acc}")

def valid(epoch, max_epoch):
    print(f"Valid Epoch [{epoch}/{max_epoch}]")
    model.eval() # Model to evaluation mode

    valid_loss = 0
    correct = 0
    total = 0

    for idx, (x, y) in enumerate(valid_dataloader):
        x = x.to(device) # maps data into GPU memory
        y = y.to(device)

        with torch.no_grad():
            y_pred = model(x) # model inference
            valid_loss += criterion(y_pred, y).item()

            _, inference = y_pred.max(1)

            total += x.size(0)
            correct += inference.eq(y).sum().item()

    acc = 100*correct/total
    print(f"Epoch [{epoch}/{max_epoch}] Valid Loss: {valid_loss/total} Valid Accuracy: {acc}")

    if not os.path.exists('checkpoint'):
        os.mkdir('checkpoint')

    torch.save(model.state_dict(), f'checkpoint/{model_path}')
    print(f"Epoch [{epoch}/{max_epoch}] Valid Model Saved: checkpoint/{model_path}")

# Custom leraning rate scheduler is use.
# Decay learning rate by 10 at epoch 5.
def lr_schedule(optimizer, epoch):
    if epoch == 5:
        lr = learning_rate / 10
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

### **Data Preparation**
* In this **Tutorial**, we are going to use **torchvision** which contains famous vision dataset, and we will use **CIFAR-10** especially 
* **CIFAR-10** is dataset for classifying image into **10** different categories.
* The **10** different classes represent **airplanes**, **cars**, **birds**, **cats**, **deer**, **dogs**, **frogs**, **horses**, **ships**, and **trucks**. 
* There are **6,000** images of each class.


In [8]:
import torchvision
import torchvision.transforms as transforms

from torch.utils.data import DataLoader

transform = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
valid_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


### **Training**

In [9]:
for epoch in range(0, max_epoch):
  lr_schedule(optimizer, epoch)
  train(epoch, max_epoch)
  valid(epoch, max_epoch)

Train Epoch [0/10]
Epoch [0/10] Batch [0] Train Loss: 2.2806129455566406
Epoch [0/10] Batch [100] Train Loss: 1.8704365491867065
Epoch [0/10] Batch [200] Train Loss: 1.5759059190750122
Epoch [0/10] Batch [300] Train Loss: 1.4987863302230835
Epoch [0/10] Train Loss: 0.012912284767627716 Train Accuracy: 39.414
Valid Epoch [0/10]
Epoch [0/10] Valid Loss: 0.010671390163898469 Valid Accuracy: 50.39
Epoch [0/10] Valid Model Saved: checkpoint/densenet121_cifar10.pt
Train Epoch [1/10]
Epoch [1/10] Batch [0] Train Loss: 1.4521450996398926
Epoch [1/10] Batch [100] Train Loss: 1.0664825439453125
Epoch [1/10] Batch [200] Train Loss: 1.0352414846420288
Epoch [1/10] Batch [300] Train Loss: 0.9708278775215149
Epoch [1/10] Train Loss: 0.008630217782258988 Train Accuracy: 60.538
Valid Epoch [1/10]
Epoch [1/10] Valid Loss: 0.008917009031772614 Valid Accuracy: 60.44
Epoch [1/10] Valid Model Saved: checkpoint/densenet121_cifar10.pt
Train Epoch [2/10]
Epoch [2/10] Batch [0] Train Loss: 0.8301960825920105
E